In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
%load_ext line_profiler

In [2]:
import os
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import matplotlib
matplotlib.rcParams.update({'font.size': 16})

from euclid_morphology import cutouts, inference

2022-10-23 14:44:03.971467: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-23 14:44:04.137075: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-10-23 14:44:04.141333: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-23 14:44:04.141348: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if yo

In [3]:
repo_root = Path(os.path.dirname(os.getcwd()))
data_dir = repo_root / 'data/example_tile'
data_dir

PosixPath('/nvme1/scratch/walml/repos/zoobot-euclid/data/example_tile')

In [4]:
catalog_file = data_dir / 'EUC_MER_FINAL-CUTOUTS-CAT_TILE100158586-2F9FF9_20220829T221845.491503Z_00.00.fits'
mosaic_file = data_dir / 'EUC_MER_BGSUB-MOSAIC-VIS_TILE100158586-863FA9_20220829T190315.054985Z_00.00.fits'
segmentation_file = data_dir / 'EUC_MER_FINAL-SEGMAP_TILE100158586-CB5786_20220829T221845.491530Z_00.00.fits'


In [5]:
%%timeit
image, seg, catalog = cutouts.load_data_for_mosaic(catalog_file, segmentation_file, mosaic_file)

7.98 s ± 67.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
image, seg, catalog = cutouts.load_data_for_mosaic(catalog_file, segmentation_file, mosaic_file)

In [7]:
len(catalog)


21529

In [8]:
source_index = 400

In [9]:
%%timeit
cutout = cutouts.prepare_image(image, seg, catalog, source_index, mode='seg', m=1.5)

1.01 s ± 1.46 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
cutout = cutouts.prepare_image(image, seg, catalog, source_index, mode='seg', m=1.5)

In [11]:
%%timeit
cutout_maj = cutouts.prepare_image(image, seg, catalog, source_index, mode='maj', m=1.5)
# don't use this later for ML testing

1.68 s ± 27.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [13]:
%%timeit
model_path = '../models/zoobot_example.tflite'
prediction = inference.load_and_predict(cutout, model_path)

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


28.9 ms ± 64.4 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


0        100158586000084
1        100158586000086
2        100158586000095
3        100158586000096
4        100158586000098
              ...       
21524    100158586119142
21525    100158586119163
21526    100158586119180
21527    100158586119269
21528    100158586119322
Name: SEG_ID, Length: 21529, dtype: int64

Loading all data (including segmap, which may be optional) takes 8 seconds for 20k sources.
Less than a ms per source.

Creating cutouts takes 1 second (segmap) or 1.7 seconds (ellipse). **By far the dominant time cost is the cutouts.**

Making predictions takes ~30ms on single image, including loading model. So it's probably not necessary to worry about batching.


## Cutout Timing

1.1 s total for prepare image (segmap mode). Where does that come from?

In [21]:
seg_ids = catalog['SEG_ID'].values

In [22]:
%%timeit
cutouts.get_cutout_from_seg_map(seg, image, np.random.choice(seg_ids))

879 ms ± 13.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Mostly from within get_cutout_from_seg_map. This is 
- finding the source index (slow, I assume)
- selecting square cutout indices (fast, I assume)
- slicing and returning (fast, I assume)

In [23]:
%%timeit
seg_idxs = np.where(seg == np.random.choice(seg_ids))

863 ms ± 1.45 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Yep, the vast majority of the time is the np.where call finding which object to look up.